In [1]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

import matplotlib.pyplot as plt

In [2]:
base_dir = 'C:/Users/user'  # 경로

excel_file = 'HYUNDAI2020-8-2514시 50분_44초 merging.xlsx'      # 파일명 

excel_dir = os.path.join(base_dir, excel_file)


df=pd.read_excel(excel_dir, header=0)
df

,Unnamed: 0,date,title,source,contents,link
0,0,3분,"팀 쿡 애플 CEO, 3304억원 스톡옵션 받는다",아시아경제,"제프 베이조스 아마존 CEO, 마크 저커버그 페이스북 CEO, 일론 머스크 테슬라 ...",https://view.asiae.co.kr/article/2020082514493...
1,1,6분,중국 전기차시장 1년 만에 성장세로 전환,ZDNet Korea,업체별로는 BEV·PHEV 1·2위 업체인 상하이GM울링과 테슬라 등이 판매 호조를...,https://zdnet.co.kr/view/?no=20200825132655
2,2,17분,"코로나19 시대에도 살아남을 혁신 기술기업은? [더 나은 세계, SDGs] (148)",세계일보,지난 20일 CNN은 미국에서 7대 기술주(마이크로소프트·아마존·구글 알파벳·애플·...,http://www.segye.com/content/html/2020/08/25/2...
3,3,18분,국내 주식 `소수점 매매` 도입…`동학개미` 화력 증강되나,이데일리언론사 선정,‘테슬라’ 산 국내 개인 투자자는 테슬라 주주가 아니다? 25일 금융당국과 금융투자...,http://www.edaily.co.kr/news/newspath.asp?news...
4,4,18분,배터리 신사업까지 도전…中 정부 업은 CATL의 굴기,디지털타임스,테슬라를 필두로 세계적인 완성차 업체들과 협력을 확대하는 동시에 사업 저변까지 넓히...,http://www.dt.co.kr/contents.html?article_no=2...
...,...,...,...,...,...,...
995,995,5일,"하나은행, ‘뉴 하나원큐’ 선봬",CNB뉴스,하나은행은 ‘뉴 하나원큐’ 출시를 기념해 10월말까지 앱을 설치하고 얼굴인증 또는 ...,http://www.cnbnews.com/news/article.html?no=45...
996,996,5일,"일동제약, 유엔SDGs협회 선정 ‘글로벌지속가능기업’ 등재",로이슈,"일동제약은 구글, 인텔, 나이키, GM, 테슬라, 이케아, 맥킨지, JP모건, 코카...",http://www.lawissue.co.kr/view.php?ud=20200820...
997,997,5일,"하나은행, 모바일 금융시장 혁신 이끈다…'뉴 하나원큐' 출사표",그린포스트코리아,"더불어 추석까지 이벤트에 참여하면 추첨을 통해 테슬라, 뉴 카니발 등의 자동차와 냉...",http://www.greenpostkorea.co.kr/news/articleVi...
998,998,5일,우리는 왜 디지털 전환(Digital Transformation)을 서둘러야 할까,내외뉴스통신,"이와 맞물려 구글, 애플, 마이크로소프트(MS), 아마존, 넷플릭스, 테슬라, 줌 ...",http://www.nbnnews.co.kr/news/articleView.html...


In [3]:

x=df['title']
x

0                          팀 쿡 애플 CEO, 3304억원 스톡옵션 받는다
1                               중국 전기차시장 1년 만에 성장세로 전환
2      코로나19 시대에도 살아남을 혁신 기술기업은? [더 나은 세계, SDGs] (148)
3                     국내 주식 `소수점 매매` 도입…`동학개미` 화력 증강되나
4                        배터리 신사업까지 도전…中 정부 업은 CATL의 굴기
                            ...                       
995                                  하나은행, ‘뉴 하나원큐’ 선봬
996                   일동제약, 유엔SDGs협회 선정 ‘글로벌지속가능기업’ 등재
997                 하나은행, 모바일 금융시장 혁신 이끈다…'뉴 하나원큐' 출사표
998      우리는 왜 디지털 전환(Digital Transformation)을 서둘러야 할까
999                        시속 402km 슈퍼 전기차, 빠른 가속력이 매력
Name: title, Length: 1000, dtype: object

In [4]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
OOV_INDEX = 1

# 데이터 타입
#ENCODER_INPUT  = 0
#DECODER_INPUT  = 1
#DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수 
max_sequences = 100
max_words=100
# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("=[.,!?\"':;~()]")


In [5]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", str(sentence))
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

# 기사 제목만 마이닝

In [6]:
x_new = pos_tag(x)

sentences = []
sentences.extend(x_new)


words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, OOV]

#print(words) # words : test,trainset의 모든 단어들이 들어간 list

# 단어와 인덱스의 단어사전 생성
word_to_index = {word: index for index, word in enumerate(words)}  # 단어 : 인덱스값 (인코딩시 사용)  # 단어 : 인덱스값
index_to_word = {index: word for index, word in enumerate(words)}  # 인덱스값 : 단어 (디코딩시 사용)  # 인덱스값 : 단어

word_to_index


{'<PADDING>': 0,
 '<OOV>': 1,
 '호출': 2,
 '2천만원': 3,
 '잇': 4,
 '키움': 5,
 '공급': 6,
 '충전기': 7,
 '256': 8,
 '품은': 9,
 '쇼핑': 10,
 'OK': 11,
 '가이드': 12,
 ')"': 13,
 '강태선': 14,
 '㎞': 15,
 '지주': 16,
 '도주': 17,
 '자율': 18,
 '만들': 19,
 '수석': 20,
 'MAGAT': 21,
 '운영': 22,
 '학': 23,
 '늘며': 24,
 '임금': 25,
 '악화': 26,
 '폭락': 27,
 '산악인': 28,
 '상륙': 29,
 '家': 30,
 '영상': 31,
 '눈길': 32,
 '아니네': 33,
 '코인': 34,
 '(': 35,
 '첨단': 36,
 '100만': 37,
 '택했': 38,
 '협업': 39,
 '제강': 40,
 '아기': 41,
 '차질': 42,
 '제어': 43,
 '뻔했던': 44,
 '정부': 45,
 '달해': 46,
 '장사': 47,
 '정은경': 48,
 '업': 49,
 '있을': 50,
 '부캐': 51,
 '경기': 52,
 '방천': 53,
 '성남': 54,
 '포스트': 55,
 '만든': 56,
 '기고': 57,
 '빚': 58,
 '20분': 59,
 '파나소닉': 60,
 '담았다': 61,
 '합계': 62,
 '되려면': 63,
 '청정': 64,
 '궁금타': 65,
 '이룬': 66,
 '차통': 67,
 '환의': 68,
 '스톡': 69,
 '될까': 70,
 '털썩': 71,
 '시속': 72,
 'Biz': 73,
 '보도': 74,
 '부양책': 75,
 '매': 76,
 '힘들': 77,
 '"…': 78,
 '쑥': 79,
 'SNS': 80,
 '하는': 81,
 '모바일': 82,
 '적극': 83,
 '완화': 84,
 '계속': 85,
 '갑론': 86,
 '연방': 87,
 '출발': 88,
 '기구

In [7]:
res=pd.DataFrame(list(word_to_index.items()),
                   columns=['단어', '빈도수'])
res.to_excel('hyundai_텍스트마이닝_제목만.xlsx')